# ESTIMATE IMPACT TO OD MATRIX
### Chọn ra cặp OD điển hình để khảo sát tỉ lệ chuyển đôi, phân vị thời gian .... 

In [1]:
import yaml
import pandas as pd
import numpy as np
from lxml import etree
import seaborn as sns
import matplotlib.pyplot as plt

import shutil
import os
import csv

# 0. Set up các file

#### Simple test

In [2]:
# TRANSIT_VEHICLE_PATH = "data\\simple_scenario\\transitVehicles.xml"
# EVENT_PATH = "data\\simple_scenario\\output\\output_events.xml"
# PLAN_PATH = "data\\simple_scenario\\plans.xml"
# NETWORK_PATH = "data\\simple_scenario\\network.xml"
# OUTPUT_EVENT_AFTER_PROCESSOR = "data\\simple_scenario\\scoring\\estimate_impact_to_OD\\trip_from_zone_to_zone.csv"

# OUTPUT_RESULT_FOLDER = "data\\simple_scenario\\scoring\\estimate_impact_to_OD\\visaulize"
# OUTPUT_EVENT_AFTER_PROCESSOR_IN_DRAW_MODE = OUTPUT_RESULT_FOLDER + "\\trip_from_zone_to_zone_draw_mode.csv"

### After Scenario

In [3]:
# TRANSIT_VEHICLE_PATH = "data\\after\\output_transitVehicles.xml"
# EVENT_PATH = "data\\after\\output_events.xml"
# PLAN_PATH = "data\\after\\plans_scale0.375true.xml"
# NETWORK_PATH = "data\\after\\network.xml"
# OUTPUT_EVENT_AFTER_PROCESSOR = "data\\after\\scoring\\estimate_impact_to_OD\\trip_from_zone_to_zone.csv"

# OUTPUT_RESULT_FOLDER = "data\\after\\scoring\\estimate_impact_to_OD\\visaulize"
# OUTPUT_EVENT_AFTER_PROCESSOR_IN_DRAW_MODE = OUTPUT_RESULT_FOLDER + "\\trip_from_zone_to_zone_draw_mode.csv"

# SCHEDULE_PATH = "data\\after\\output_transitSchedule.xml"

# NAME_SCENARIO = "AFTER IMPROVEMENT"  # Change this value accordingly

### Before Scenario 

In [4]:
TRANSIT_VEHICLE_PATH = "data\\before\\output_transitVehicles.xml"
EVENT_PATH = "data\\before\\output_events.xml"
PLAN_PATH = "data\\before\\plans_scale0.375true.xml"
NETWORK_PATH = "data\\before\\network.xml"
OUTPUT_EVENT_AFTER_PROCESSOR = "data\\before\\scoring\\estimate_impact_to_OD\\trip_from_zone_to_zone.csv"
SCHEDULE_PATH = "data\\before\\output_transitSchedule.xml"

OUTPUT_RESULT_FOLDER = "data\\before\\scoring\\estimate_impact_to_OD\\visaulize"
OUTPUT_EVENT_AFTER_PROCESSOR_IN_DRAW_MODE = OUTPUT_RESULT_FOLDER + "\\trip_from_zone_to_zone_draw_mode.csv"
NAME_SCENARIO = "BEFORE IMPROVEMENT"  # Change this value accordingly

### Real Scenario

In [5]:
# TRANSIT_VEHICLE_PATH = "data\\real\\output_transitVehicles.xml"
# EVENT_PATH = "data\\real\\output_events.xml"
# PLAN_PATH = "data\\real\\plans_scale0.375truelans.xml"
# NETWORK_PATH = "data\\real\\network.xml"
# OUTPUT_EVENT_AFTER_PROCESSOR = "data\\real\\scoring\\estimate_impact_to_OD\\trip_from_zone_to_zone.csv"

# OUTPUT_RESULT_FOLDER = "data\\real\\scoring\\estimate_impact_to_OD\\visaulize"
# OUTPUT_EVENT_AFTER_PROCESSOR_IN_DRAW_MODE = OUTPUT_RESULT_FOLDER + "\\trip_from_zone_to_zone_draw_mode.csv"

In [6]:
if os.path.exists(OUTPUT_EVENT_AFTER_PROCESSOR):
    os.remove(OUTPUT_EVENT_AFTER_PROCESSOR)

folder_path = os.path.dirname(OUTPUT_EVENT_AFTER_PROCESSOR)
os.makedirs(folder_path, exist_ok=True)

if os.path.exists(OUTPUT_RESULT_FOLDER):
    shutil.rmtree(OUTPUT_RESULT_FOLDER)

os.makedirs(OUTPUT_RESULT_FOLDER, exist_ok=True)


## 1. Tạo class zone và point và kiểm tra xem 1 point có nằm trong zone ko

In [7]:
class Point:
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

class Zone:
    def __init__(self, zone_id, boundary_points: list[Point]):
        self.zone_id = zone_id
        self.boundary = boundary_points

    def print_boundary(self):
        print(f"Zone ID: {self.zone_id}")
        for point in self.boundary:
            print(f"({point.x}, {point.y})")

    def is_in_zone(self, point: Point) -> bool:
        """
        Kiểm tra điểm có nằm trong vùng không (bao gồm cả biên)
        Sử dụng Ray Casting Algorithm
        """
        if len(self.boundary) < 3:
            return False
        
        n = len(self.boundary)
        inside = False
        
        # Kiểm tra điểm có nằm trên biên không
        for i in range(n):
            p1 = self.boundary[i]
            p2 = self.boundary[(i + 1) % n]
            
            if self._point_on_segment(point, p1, p2):
                return True
        
        # Ray casting algorithm - bắn tia từ điểm về phía phải
        p1 = self.boundary[0]
        for i in range(1, n + 1):
            p2 = self.boundary[i % n]
            
            # Kiểm tra xem tia ngang đi qua cạnh này không
            if point.y > min(p1.y, p2.y):
                if point.y <= max(p1.y, p2.y):
                    if point.x <= max(p1.x, p2.x):
                        if p1.y != p2.y:
                            x_intersection = (point.y - p1.y) * (p2.x - p1.x) / (p2.y - p1.y) + p1.x
                        
                        if p1.x == p2.x or point.x <= x_intersection:
                            inside = not inside
            p1 = p2
        
        return inside
    
    def _point_on_segment(self, point: Point, p1: Point, p2: Point) -> bool:
        """
        Kiểm tra điểm có nằm trên đoạn thẳng p1-p2 không
        """
        # Kiểm tra điểm có nằm trong hình chữ nhật bao quanh đoạn thẳng
        if not (min(p1.x, p2.x) <= point.x <= max(p1.x, p2.x) and
                min(p1.y, p2.y) <= point.y <= max(p1.y, p2.y)):
            return False
        
        # Kiểm tra 3 điểm thẳng hàng bằng cross product
        # Cross product = 0 nghĩa là thẳng hàng
        cross = (point.y - p1.y) * (p2.x - p1.x) - (point.x - p1.x) * (p2.y - p1.y)
        
        # Dùng epsilon để xử lý sai số float
        epsilon = 1e-10
        return abs(cross) < epsilon


#### Test code kiểm tra 1 điểm trong zone

In [8]:
zone = Zone("zone1", [
        Point(0, 0),
        Point(10, 0),
        Point(10, 10),
        Point(0, 10)
    ])
    
    # Test các điểm
test_points = [
    (Point(5, 5), "Điểm ở trong"),           # True - trong vùng
    (Point(0, 0), "Điểm ở đỉnh"),            # True - trên biên (đỉnh)
    (Point(5, 0), "Điểm ở cạnh"),            # True - trên biên (cạnh)
    (Point(15, 5), "Điểm ở ngoài"),          # False - ngoài vùng
    (Point(10, 5), "Điểm ở cạnh phải"),      # True - trên biên
]
    
for point, description in test_points:
    result = zone.is_in_zone(point)
    print(f"{description} ({point.x}, {point.y}): {result}")

Điểm ở trong (5.0, 5.0): True
Điểm ở đỉnh (0.0, 0.0): True
Điểm ở cạnh (5.0, 0.0): True
Điểm ở ngoài (15.0, 5.0): False
Điểm ở cạnh phải (10.0, 5.0): True


## 2. Tạo ra simple_zone từ Hàm sinh ra Zone theo ô bàn cờ x hàng y côt dựa trên xml netwwork đưa vào

In [9]:
!powershell -Command "Get-Content 'data\\simple_scenario\\network.xml' -TotalCount 20 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

5: <?xml version="1.0" encoding="UTF-8"?>
5: <!DOCTYPE network SYSTEM "http://www.matsim.org/files/dtd/network_v1.dtd">
5: <network>
5:   <nodes>
5:     <node id="n_0_0" x="460000.0" y="5740000.0" />
5:     <node id="n_1_0" x="463000.0" y="5740000.0" />
5:     <node id="n_2_0" x="466000.0" y="5740000.0" />
5:     <node id="n_3_0" x="469000.0" y="5740000.0" />
5:     <node id="n_4_0" x="472000.0" y="5740000.0" />
5:     <node id="n_5_0" x="475000.0" y="5740000.0" />
5:     <node id="n_0_1" x="460000.0" y="5743000.0" />
5:     <node id="n_1_1" x="463000.0" y="5743000.0" />
5:     <node id="n_2_1" x="466000.0" y="5743000.0" />
5:     <node id="n_3_1" x="469000.0" y="5743000.0" />
5:     <node id="n_4_1" x="472000.0" y="5743000.0" />
5:     <node id="n_5_1" x="475000.0" y="5743000.0" />
5:     <node id="n_0_2" x="460000.0" y="5746000.0" />
5:     <node id="n_1_2" x="463000.0" y="5746000.0" />
5:     <node id="n_2_2" x="466000.0" y="5746000.0" />
5:     <node id="n_3_2" x="469000.0" y="5746

In [10]:
def generate_checkerboard_shape_zone_from_network(network_path: str, rows: int, cols: int) -> list[Zone]:
    """
    Tạo các vùng hình chữ nhật dạng bàn cờ từ mạng lưới giao thông
    network_path: đường dẫn file mạng lưới (XML)
    rows: số hàng
    cols: số cột
    Trả về danh sách các vùng Zone
    """
    tree = etree.parse(network_path)
    root = tree.getroot()
    
    x_values = []
    y_values = []
    
    for node in root.xpath('//network/nodes/node'):
        x = float(node.get('x'))
        y = float(node.get('y'))
        x_values.append(x)
        y_values.append(y)
    
    min_x, max_x = min(x_values), max(x_values)
    min_y, max_y = min(y_values), max(y_values)

    print(f"Network bounds: X({min_x}, {max_x}), Y({min_y}, {max_y})")
    
    zone_width = (max_x - min_x) / cols
    zone_height = (max_y - min_y) / rows
    
    zones = []
    
    for i in range(rows):
        for j in range(cols):
            zone_id = f"zone_{i}_{j}"
            boundary_points = [
                Point(min_x + j * zone_width, min_y + i * zone_height),
                Point(min_x + (j + 1) * zone_width, min_y + i * zone_height),
                Point(min_x + (j + 1) * zone_width, min_y + (i + 1) * zone_height),
                Point(min_x + j * zone_width, min_y + (i + 1) * zone_height)
            ]
            zones.append(Zone(zone_id, boundary_points))
    
    return zones

In [11]:
test_zones : list[Zone]= generate_checkerboard_shape_zone_from_network(NETWORK_PATH, 4, 4)
for zone in test_zones:
    zone.print_boundary()

Network bounds: X(439460.06825334975, 483552.56240213366), Y(5711222.598506039, 5766338.24313264)
Zone ID: zone_0_0
(439460.06825334975, 5711222.598506039)
(450483.1917905457, 5711222.598506039)
(450483.1917905457, 5725001.50966269)
(439460.06825334975, 5725001.50966269)
Zone ID: zone_0_1
(450483.1917905457, 5711222.598506039)
(461506.3153277417, 5711222.598506039)
(461506.3153277417, 5725001.50966269)
(450483.1917905457, 5725001.50966269)
Zone ID: zone_0_2
(461506.3153277417, 5711222.598506039)
(472529.4388649377, 5711222.598506039)
(472529.4388649377, 5725001.50966269)
(461506.3153277417, 5725001.50966269)
Zone ID: zone_0_3
(472529.4388649377, 5711222.598506039)
(483552.56240213366, 5711222.598506039)
(483552.56240213366, 5725001.50966269)
(472529.4388649377, 5725001.50966269)
Zone ID: zone_1_0
(439460.06825334975, 5725001.50966269)
(450483.1917905457, 5725001.50966269)
(450483.1917905457, 5738780.420819339)
(439460.06825334975, 5738780.420819339)
Zone ID: zone_1_1
(450483.1917905457

## 3. Tạo bảng chứa trip từng người đi từ zone nào đến zone nào trong bao lâu, phương tiện gì
### BUS/PT TRAVELTIME INCLUDE PERSON IN BUS/PT AND PERSON WALK, WAIT TO PT/BUS

### Xử lý file transit vehicle

In [12]:
!powershell -Command "Get-Content 'data\\simple_scenario\\transitVehicles.xml' -TotalCount 20 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

5: <?xml version="1.0" encoding="UTF-8"?>
5: <vehicleDefinitions xmlns="http://www.matsim.org/files/dtd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.matsim.org/files/dtd http://www.matsim.org/files/dtd/vehicleDefinitions_v1.0.xsd">
5:   <vehicleType id="bus_type"> <capacity><seats persons="50"/><standingRoom persons="0"/></capacity> <length meter="10.0"/> </vehicleType>
5:   <vehicleType id="tram_type"> <capacity><seats persons="50"/><standingRoom persons="0"/></capacity> <length meter="10.0"/> </vehicleType>
5:   <vehicleType id="train_type"> <capacity><seats persons="100"/><standingRoom persons="0"/></capacity> <length meter="10.0"/> </vehicleType>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_0" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_1" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_2" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_3" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_4" type="bus_typ

### Xử lý ra dict chứa id và type

In [13]:
tree = etree.parse(TRANSIT_VEHICLE_PATH)
root = tree.getroot()

#### xlmn trong thẻ vehicleDefinitions  là namespace cho tất cả các tag trong tag này. nghĩa là tên đầy đủ của tag là {http://www.matsim.org/files/dtd}vehicleDefinitions, {http://www.matsim.org/files/dtd}vehicle.

#### Khi dùng lxml phải dùng namespace: root.xpath("//m:vehicleDefinitions/m:vehicle", namespaces=ns)

In [14]:
ns = {'m': 'http://www.matsim.org/files/dtd'}
vehtype_dict = {}


for node in root.xpath("//m:vehicleDefinitions/m:vehicle", namespaces=ns):
    id = node.xpath("@id")[0]
    type = node.xpath("@type")[0]
    print([id,type])
    vehtype_dict[id] = type



['1_0', 'tram_93pax']
['1_1', 'tram_93pax']
['1_2', 'tram_93pax']
['1_3', 'tram_93pax']
['1_4', 'tram_93pax']
['2_0', 'tram_93pax']
['2_1', 'tram_93pax']
['2_2', 'tram_93pax']
['2_3', 'tram_93pax']
['2_4', 'tram_93pax']
['3_0', 'tram_93pax']
['3_1', 'tram_93pax']
['3_2', 'tram_93pax']
['3_3', 'tram_93pax']
['3_4', 'tram_93pax']
['4_0', 'tram_93pax']
['4_1', 'tram_93pax']
['4_2', 'tram_93pax']
['4_3', 'tram_93pax']
['4_4', 'tram_93pax']
['rb43_0', 'train']
['rb43_1', 'train']
['rb43_2', 'train']
['rb43_3', 'train']
['rb43_4', 'train']
['re11_0', 'train']
['re11_1', 'train']
['re11_2', 'train']
['re11_3', 'train']
['re11_4', 'train']
['re2_0', 'train']
['re2_1', 'train']
['re2_2', 'train']
['re2_3', 'train']
['re2_4', 'train']
['veh_bus_10_in_07:00:00', 'bus_90pax']
['veh_bus_10_in_07:15:00', 'bus_90pax']
['veh_bus_10_in_07:30:00', 'bus_90pax']
['veh_bus_10_in_07:45:00', 'bus_90pax']
['veh_bus_10_in_08:00:00', 'bus_90pax']
['veh_bus_10_in_08:15:00', 'bus_90pax']
['veh_bus_10_in_08:30:00'

### Xử lý event. Đồng thời nhận thêm zone để quan sát điểm đầu cuối của trip

In [15]:
!powershell -Command "Get-Content 'data\\simple_scenario\\output\\output_events.xml' -TotalCount 1000 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

5: <?xml version="1.0" encoding="utf-8"?>
5: <events version="1.0">
5: 	<event time="21600.0" type="TransitDriverStarts" driverId="pt_veh_Tram_Line_1_dir_fwd_0_tram_type" vehicleId="veh_Tram_Line_1_dir_fwd_0" transitLineId="Tram_Line_1" transitRouteId="dir_fwd" departureId="veh_Tram_Line_1_dir_fwd_0"  />
5: 	<event time="21600.0" type="departure" person="pt_veh_Tram_Line_1_dir_fwd_0_tram_type" link="7" legMode="car"  />
5: 	<event time="21600.0" type="PersonEntersVehicle" person="pt_veh_Tram_Line_1_dir_fwd_0_tram_type" vehicle="veh_Tram_Line_1_dir_fwd_0"  />
5: 	<event time="21600.0" type="TransitDriverStarts" driverId="pt_veh_Tram_Line_1_dir_bwd_0_tram_type" vehicleId="veh_Tram_Line_1_dir_bwd_0" transitLineId="Tram_Line_1" transitRouteId="dir_bwd" departureId="veh_Tram_Line_1_dir_bwd_0"  />
5: 	<event time="21600.0" type="departure" person="pt_veh_Tram_Line_1_dir_bwd_0_tram_type" link="74" legMode="car"  />
5: 	<event time="21600.0" type="PersonEntersVehicle" person="pt_veh_Tram_Line_

#3.1 Chỉ đếm những người lên xe bus khác lái xe

In [16]:
hint_bus_type = "bus"
schema = ['vehIdList', 'vehicleTypeList', 'mainMode' , 'travelTime', 'OZone', 'DZone', 'actstart', 'actend', 'xO', 'yO', 'xD', 'yD', 'startTime']
person_trip_map = {}
simple_zones : list[Zone]= generate_checkerboard_shape_zone_from_network(NETWORK_PATH, 20, 20)

with open(OUTPUT_EVENT_AFTER_PROCESSOR, 'a') as f:
    for x in schema[:-1]:
        f.write(x + ",")
    f.write(schema[-1] + "\n")

context = etree.iterparse(EVENT_PATH, events=('end',))
for event, elem in context:
    if elem.tag == "event":
        e_type = elem.get("type")

        if e_type == "actend":
            personId = elem.get("person")

            if personId in person_trip_map.keys():
                continue
            if personId.startswith("pt_"):
                continue
            if elem.get("actType") == "pt interaction":
                continue

            actstart = elem.get("actType")
            
            x = float(elem.get("x"))
            y = float(elem.get("y"))
            person_location = Point(x, y)
            in_zone_id = "undefined"
            for zone in simple_zones:
                if zone.is_in_zone(person_location):
                    in_zone_id = zone.zone_id
                    break

            person_trip_map[personId] = {
                "actstart" : actstart,
                "OZone": in_zone_id,
                "xO": x,
                "yO": y
                }


        elif e_type == "departure":
            personId = elem.get("person")

            # Nếu trip của người này đã được ghi nhận trước đó và chưa kết thúc thì bỏ qua
            if personId in person_trip_map.keys():
                if "vehIdList" not in person_trip_map[personId].keys():
                    person_trip_map[personId]["vehIdList"] = []
                    person_trip_map[personId]["vehicleTypeList"] = []
                    person_trip_map[personId]["mainMode"] =  elem.get("computationalRoutingMode")
                    person_trip_map[personId]["startTime"] = float(elem.get("time"))
                

        elif e_type == "PersonEntersVehicle":
            personId = elem.get("person")
            veh_id = elem.get("vehicle")

            if personId not in person_trip_map.keys():
                continue
            if personId.startswith("pt_"):
                continue
                
            person_trip_map[personId]["vehIdList"].append(veh_id)

            if veh_id in vehtype_dict.keys():
                person_trip_map[personId]["vehicleTypeList"].append(vehtype_dict[veh_id])
            else :
                person_trip_map[personId]["vehicleTypeList"].append("undefined")


        elif e_type == "actstart":
            personId = elem.get("person")

            if personId not in person_trip_map.keys():
                continue
            if personId.startswith("pt_"):
                continue
            if elem.get("actType") == "pt interaction":
                continue
        
            person_trip_map[personId]["travelTime"] = float(elem.get("time")) - person_trip_map[personId]["startTime"]
            person_trip_map[personId]["actend"] = elem.get("actType")

            # Xác định vùng DZone chứa ddierm cuối của trip
            xD = float(elem.get("x"))
            yD = float(elem.get("y"))
            person_location = Point(xD, yD)
            in_zone_id = "undefined"
            for zone in simple_zones:
                if zone.is_in_zone(person_location):
                    in_zone_id = zone.zone_id
                    break
            person_trip_map[personId]["DZone"] = in_zone_id

            # Nếu người này chỉ đi bộ nhưng vẫn có computationalRoutingMode là pt/car thì bỏ qua
            # if len(person_trip_map[personId]["vehIdList"]) == 0 or len(person_trip_map[personId]["vehicleTypeList"]) == 0:
            #     del person_trip_map[personId]
        
            # else:
            #     with open(OUTPUT_EVENT_AFTER_PROCESSOR, 'a', newline='') as f:
            #         writer = csv.writer(f)
                    
            #         # Chuẩn bị dữ liệu
            #         # Biến list thành chuỗi, ví dụ dùng dấu ';' để phân cách các ID bên trong
            #         veh_ids = ";".join(map(str, person_trip_map[personId]['vehIdList']))
            #         veh_types = ";".join(map(str, person_trip_map[personId]['vehicleTypeList']))

            #         schema = ['vehIdList', 'vehicleTypeList', 'mainMode' , 'travelTime', 'OZone', 'DZone', 'actstart', 'actend', 'startTime']
                    
            #         row = [
            #             veh_ids,
            #             veh_types,
            #             person_trip_map[personId]['mainMode'],
            #             person_trip_map[personId]['travelTime'],
            #             person_trip_map[personId]['OZone'],
            #             person_trip_map[personId]['DZone'],
            #             person_trip_map[personId]['actstart'],
            #             person_trip_map[personId]['actend'],
            #             person_trip_map[personId]['startTime']
                        
            #         ]
                    
            #         # Ghi cả dòng một lần duy nhất
            #         writer.writerow(row)
            #     del person_trip_map[personId]
            with open(OUTPUT_EVENT_AFTER_PROCESSOR, 'a', newline='') as f:
                writer = csv.writer(f)
                
                # Chuẩn bị dữ liệu
                # Biến list thành chuỗi, ví dụ dùng dấu ';' để phân cách các ID bên trong
                veh_ids = ";".join(map(str, person_trip_map[personId]['vehIdList']))
                veh_types = ";".join(map(str, person_trip_map[personId]['vehicleTypeList']))

                schema = ['vehIdList', 'vehicleTypeList', 'mainMode' , 'travelTime', 'OZone', 'DZone', 'actstart', 'actend', 'xO', 'yO', 'xD', 'yD', 'startTime']
                
                row = [
                    veh_ids,
                    veh_types,
                    person_trip_map[personId]['mainMode'],
                    person_trip_map[personId]['travelTime'],
                    person_trip_map[personId]['OZone'],
                    person_trip_map[personId]['DZone'],
                    person_trip_map[personId]['actstart'],
                    person_trip_map[personId]['actend'],
                    person_trip_map[personId]['xO'],
                    person_trip_map[personId]['yO'],
                    xD,
                    yD,
                    person_trip_map[personId]['startTime']
                    
                ]
                
                # Ghi cả dòng một lần duy nhất
                writer.writerow(row)
            del person_trip_map[personId]

    elem.clear()

print("Finished processing travel data.")






Network bounds: X(439460.06825334975, 483552.56240213366), Y(5711222.598506039, 5766338.24313264)
Finished processing travel data.


# Vẽ đồ thị

In [17]:
# --- BƯỚC 1: XỬ LÝ DỮ LIỆU & TẠO FILE DEBUG ---
df = pd.read_csv(OUTPUT_EVENT_AFTER_PROCESSOR)
df.columns = df.columns.str.strip()

# Chuẩn hóa chuỗi (xóa khoảng trắng, xử lý giá trị rỗng)
for col in ['vehIdList', 'vehicleTypeList', 'mainMode', 'OZone', 'DZone']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().replace(['nan', 'None', 'undefined', ''], '')

def get_drawmode(row):
    vid, vtype = row['vehIdList'], row['vehicleTypeList']
    mmode = row['mainMode'].lower()
    
    if not vid and not vtype: return 'walk'
    if mmode == 'car': return 'car'
    if mmode == 'pt':
        return 'bus' if 'bus' in vtype.lower() else 'pt not bus'
    return mmode if mmode else 'unknown'

df['drawmode'] = df.apply(get_drawmode, axis=1)
df['travelTime'] = pd.to_numeric(df['travelTime'], errors='coerce')
# --- NEW: Convert startTime to numeric for Item C processing ---
df['startTime'] = pd.to_numeric(df['startTime'], errors='coerce')

# Lưu file debug
df.to_csv(OUTPUT_EVENT_AFTER_PROCESSOR_IN_DRAW_MODE, index=False)

# --- BƯỚC 2: HÀM VẼ BÁO CÁO TỔNG HỢP ---
def generate_final_report(data_df, folder_name):
    if os.path.exists(folder_name): shutil.rmtree(folder_name)
    os.makedirs(folder_name)
    
    def create_combined_fig(subset_in, title, filename):
        subset = subset_in.copy() # Avoid SettingWithCopyWarning
        
        # --- C. PREPARE DATA FOR TIME ANALYSIS ---
        if 'startTime' in subset.columns:
            subset['start_hour'] = subset['startTime'] / 3600
        else:
             subset['start_hour'] = 0 # Fallback

        # UPDATE: Change layout to 3x2 to include Time analysis (Item C)
        fig, axes = plt.subplots(3, 2, figsize=(18, 20)) 
        fig.suptitle(f"{title}\n(Tổng cộng: {len(subset)} Trip)", fontsize=22, fontweight='bold', y=0.98)
        
        # 1. Tỷ lệ phương tiện
        mode_counts = subset['drawmode'].value_counts()
        axes[0, 0].pie(mode_counts, labels=mode_counts.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
        axes[0, 0].set_title("Tỷ lệ phương tiện", fontsize=16)
        
        # 2. Thời gian trung bình (Modified for Item A & B.1)
        avg_times = subset.groupby('drawmode')['travelTime'].mean().sort_values(ascending=False)
        
        # Fix Item A: Added hue and legend=False to silence warning
        sns.barplot(x=avg_times.index, y=avg_times.values, ax=axes[0, 1], palette='viridis', hue=avg_times.index, legend=False)
        
        for i, mode in enumerate(avg_times.index):
            v = avg_times[mode]
            count = mode_counts.get(mode, 0) 
            # Item B.1: Add n=... to text
            axes[0, 1].text(i, v, f'{v:.1f}\n(n={count})', ha='center', va='bottom', fontweight='bold')
        axes[0, 1].set_title("Thời gian di chuyển trung bình (giây)", fontsize=16)
        
        # 3 & 4. Boxplots Bus và Car tách biệt
        for idx, mode_name, color in [( (1,0), 'bus', 'skyblue'), ((1,1), 'car', 'salmon')]:
            m_data = subset[subset['drawmode'] == mode_name]['travelTime']
            ax = axes[idx]
            if not m_data.empty:
                sns.boxplot(y=m_data, ax=ax, color=color, showfliers=False)
                q1, med, q3 = np.percentile(m_data, [25, 50, 75])
                ax.text(0, med, f'Med: {med:.1f}', ha='center', fontweight='bold', bbox=dict(facecolor='white', alpha=0.5))
                ax.text(0.1, q1, f'25%: {q1:.1f}', color='blue', fontweight='bold')
                ax.text(0.1, q3, f'75%: {q3:.1f}', color='red', fontweight='bold')
                ax.set_title(f"Phân vị: {mode_name.upper()} (N={len(m_data)})", fontsize=16)
            else:
                ax.text(0.5, 0.5, f"Không có dữ liệu {mode_name.upper()}", ha='center')

        # --- C. ITEM SUGGESTION: DEPARTURE TIME PLOTS ---
        # 5. Departure Time Distribution (Histogram)
        ax_hist = axes[2, 0]
        sns.histplot(data=subset, x='start_hour', hue='drawmode', kde=True, ax=ax_hist, element="step", common_norm=False)
        ax_hist.set_title("Phân bố giờ khởi hành (Departure Time)", fontsize=16)
        ax_hist.set_xlabel("Hour of Day")
        ax_hist.set_ylabel("Count")
        
        # 6. Average Travel Time by Hour (Line Plot)
        ax_line = axes[2, 1]
        subset['hour_int'] = subset['start_hour'].astype(int)
        # Group by hour and mode
        time_trend = subset.groupby(['hour_int', 'drawmode'])['travelTime'].mean().reset_index()
        if not time_trend.empty:
            sns.lineplot(data=time_trend, x='hour_int', y='travelTime', hue='drawmode', ax=ax_line, marker='o')
        ax_line.set_title("Biến động thời gian di chuyển theo giờ", fontsize=16)
        ax_line.set_xlabel("Hour of Day")
        ax_line.set_ylabel("Avg Travel Time (s)")
        ax_line.grid(True, linestyle='--', alpha=0.7)

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig(os.path.join(folder_name, filename))
        plt.close()

    # Tạo ảnh Tổng thể và Top 3 OD
    create_combined_fig(df, f"BÁO CÁO TỔNG QUAN HỆ THỐNG {NAME_SCENARIO}", "00_global_summary.png")
    od_counts = df.groupby(['OZone', 'DZone']).size().reset_index(name='n').sort_values('n', ascending=False)
    for i, (_, row) in enumerate(od_counts.head(10).iterrows(), 1):
        o, d = row['OZone'], row['DZone']
        create_combined_fig(df[(df['OZone']==o) & (df['DZone']==d)], f"TOP {i} OD: {o} -> {d} {NAME_SCENARIO}", f"top_{i}_OD_{o}_{d}_{NAME_SCENARIO}.png")

generate_final_report(OUTPUT_EVENT_AFTER_PROCESSOR, OUTPUT_RESULT_FOLDER)


In [18]:
import matplotlib.patches as patches
from matplotlib.collections import LineCollection
from matplotlib.path import Path

def draw_top_5_od_map_v11(network_path, schedule_path, zones_list, data_df, output_folder, grid_info="20x20"):
    print("Generating V11 Top 5 OD Map (Refined Colors & Transparency)...")
    
    # 1. Parse Network
    tree = etree.parse(network_path)
    root = tree.getroot()
    nodes = {}
    for node in root.xpath('//network/nodes/node'):
        nodes[node.get('id')] = (float(node.get('x')), float(node.get('y')))
    links = {}
    for link in root.xpath('//network/links/link'):
        links[link.get('id')] = (link.get('from'), link.get('to'))

    # 2. Parse Bus Links
    bus_link_ids = set()
    if os.path.exists(schedule_path):
        try:
            tree_sched = etree.parse(schedule_path)
            root_sched = tree_sched.getroot()
            for route in root_sched.xpath('//transitRoute'):
                mode = route.find('transportMode')
                if mode is not None and mode.text.strip().lower() == 'bus':
                     r_path = route.find('route')
                     if r_path is not None:
                         for link_ref in r_path.xpath('link'):
                             bus_link_ids.add(link_ref.get('refId'))
        except Exception as e:
            print(f"Error parsing schedule: {e}")
    print(f"Found {len(bus_link_ids)} unique bus links.")

    # 3. Data Prep
    valid_df = data_df.copy()
    for col in ['xO', 'yO', 'xD', 'yD']:
        if col in valid_df.columns:
            valid_df[col] = pd.to_numeric(valid_df[col], errors='coerce')
    plot_df = valid_df.dropna(subset=['xO', 'yO', 'xD', 'yD'])
    ranking_df = valid_df[(valid_df['OZone'] != 'undefined') & (valid_df['DZone'] != 'undefined')]
    top_od_counts = ranking_df.groupby(['OZone', 'DZone']).size().reset_index(name='count').sort_values('count', ascending=False).head(5)

    if top_od_counts.empty:
        print("No valid OD pairs found.")
        return

    # 4. Setup Plot
    fig, ax = plt.subplots(figsize=(24, 24), facecolor='black')
    ax.set_facecolor('black')
    ax.set_aspect('equal')

    # Z-Orders
    Z_BASE = 1
    Z_POINTS = 2
    Z_BUS = 3
    Z_ZONES = 4
    Z_ARROWS = 5

    # Layer 1: Background Network
    base_lines = []
    for l_id, (u, v) in links.items():
        if u in nodes and v in nodes:
            base_lines.append([nodes[u], nodes[v]])
    lc_base = LineCollection(base_lines, colors='#cccccc', linewidths=0.5, alpha=0.3, zorder=Z_BASE)
    ax.add_collection(lc_base)

    # Layer 2: Trip Points (Refined Colors)
    # Origin: Light Blue (#ADD8E6)
    # Destination: Medium Pink (#FF69B4)
    ax.scatter(plot_df['xO'], plot_df['yO'], c='#4ec6ed', s=10, alpha=0.6, label='Origins', edgecolors='none', zorder=Z_POINTS)
    ax.scatter(plot_df['xD'], plot_df['yD'], c='#f03793', s=10, alpha=0.6, label='Destinations', edgecolors='none', zorder=Z_POINTS)

    # Layer 3: Bus Network (Red)
    bus_lines = []
    for l_id in bus_link_ids:
        if l_id in links:
            u, v = links[l_id]
            if u in nodes and v in nodes:
                bus_lines.append([nodes[u], nodes[v]])
    if bus_lines:
        lc_bus = LineCollection(bus_lines, colors='#f2f218', linewidths=2.5, alpha=1, zorder=Z_BUS)
        ax.add_collection(lc_bus)

    # Bounds
    rank_colors = ['#FF3333', '#FF9933', '#FFFF33', '#33FF33', '#33FFFF']
    zone_map = {z.zone_id: z for z in zones_list}
    unique_zones = set(top_od_counts['OZone']).union(set(top_od_counts['DZone']))
    
    min_x, max_x = float('inf'), float('-inf')
    min_y, max_y = float('inf'), float('-inf')
    for z_id in unique_zones:
        zone = zone_map.get(z_id)
        if zone:
            xs = [p.x for p in zone.boundary]
            ys = [p.y for p in zone.boundary]
            min_x, max_x = min(min_x, min(xs)), max(max_x, max(xs))
            min_y, max_y = min(min_y, min(ys)), max(max_y, max(ys))

    # Layer 4: Zones (Increased Transparency)
    # Previous ~0.9 or default. Now 0.3 (Double transparency relative to opaque-ish)
    for z_id in unique_zones:
        zone = zone_map.get(z_id)
        if not zone: continue
        # facecolor was #222222, now adding alpha=0.3
        poly = patches.Polygon([(p.x, p.y) for p in zone.boundary], closed=True, 
                               facecolor='#222222', edgecolor='white', alpha=0.6, linewidth=2.0, zorder=Z_ZONES)
        ax.add_patch(poly)
        xs = [p.x for p in zone.boundary]
        ys = [p.y for p in zone.boundary]
        ax.text(min(xs)+50, max(ys)-50, z_id, color='white', fontsize=11, fontweight='bold', ha='left', va='top', zorder=Z_ZONES+0.1)

    # Layer 5: Arrows
    cell_text = []
    table_colors = []
    
    for idx, (i, row) in enumerate(top_od_counts.iterrows()):
        color = rank_colors[idx % 5]
        o_id, d_id = row['OZone'], row['DZone']
        
        o_z, d_z = zone_map.get(o_id), zone_map.get(d_id)
        if not o_z or not d_z: continue
        
        ox = sum(p.x for p in o_z.boundary)/len(o_z.boundary)
        oy = sum(p.y for p in o_z.boundary)/len(o_z.boundary)
        dx = sum(p.x for p in d_z.boundary)/len(d_z.boundary)
        dy = sum(p.y for p in d_z.boundary)/len(d_z.boundary)
        
        if o_id == d_id:
            z_xs = [p.x for p in o_z.boundary]
            z_width = max(z_xs) - min(z_xs)
            r = z_width * 0.25 
            theta = np.linspace(np.radians(30), np.radians(330), 50)
            arc_xs = ox + r * np.cos(theta)
            arc_ys = oy + r * np.sin(theta)
            verts = list(zip(arc_xs, arc_ys))
            p = Path(verts)
            arrow = patches.FancyArrowPatch(path=p, arrowstyle='-|>', color=color, lw=3, mutation_scale=20, zorder=Z_ARROWS)
            ax.add_patch(arrow)
        else:
            rad = 0.2 + (0.05 * idx)
            arrow = patches.FancyArrowPatch((ox, oy), (dx, dy), connectionstyle=f"arc3,rad={rad}", 
                                            arrowstyle='-|>', color=color, lw=3, mutation_scale=25, zorder=Z_ARROWS)
            ax.add_patch(arrow)
        
        cell_text.append([o_id, d_id, str(row['count'])])
        table_colors.append([color, color, '#333333'])

    if min_x != float('inf'):
        pad_x = (max_x - min_x) * 0.2
        pad_y = (max_y - min_y) * 0.2
        ax.set_xlim(min_x - pad_x, max_x + pad_x)
        ax.set_ylim(min_y - pad_y, max_y + pad_y)
        
    ax.set_title(f"TOP 5 OD FLOW: {NAME_SCENARIO}", fontsize=24, color='white', pad=20)
    ax.text(0.02, 0.98, f"Grid: {grid_info}\nYellow: Bus Routes\nPoints: O=Light Blue, D=Medium Pink", transform=ax.transAxes, 
            color='white', fontsize=12, fontweight='bold', va='top', 
            bbox=dict(facecolor='black', alpha=0.5, edgecolor='none'))
    ax.axis('off')

    if cell_text:
        the_table = ax.table(cellText=cell_text,
                             colLabels=["Origin", "Destination", "Trips"],
                             cellColours=table_colors,
                             cellLoc='center',
                             loc='upper right',
                             bbox=[0.72, 0.85, 0.26, 0.13]) 
        the_table.auto_set_font_size(False)
        the_table.set_fontsize(9)
        for (r, c), cell in the_table.get_celld().items():
             cell.set_edgecolor('white')
             cell.set_linewidth(0.5)
             if r == 0:
                 cell.set_facecolor('#dddddd')
                 cell.set_text_props(weight='bold', color='black')
             else:
                 if c < 2:
                     cell.set_text_props(weight='bold', color='black')
                 else:
                     cell.set_text_props(color='white')

    save_path = os.path.join(output_folder, f"Top5_OD_Vis_Final_v11_{NAME_SCENARIO}.png")
    plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='black')
    plt.close()
    print(f"Map saved to: {save_path}")

draw_top_5_od_map_v11(NETWORK_PATH, SCHEDULE_PATH, simple_zones, df, OUTPUT_RESULT_FOLDER, grid_info="20x20")


Generating V11 Top 5 OD Map (Refined Colors & Transparency)...
Found 912 unique bus links.
Map saved to: data\before\scoring\estimate_impact_to_OD\visaulize\Top5_OD_Vis_Final_v11_BEFORE IMPROVEMENT.png
